### Load

In [1]:
# -*- coding: utf-8 -*-

from proj1_helpers import load_csv_data, create_csv_submission, predict_labels
import numpy as np
from cleaner import clean_input_data, concatenate_log
from implementations import build_poly, logistic_regression, compute_loss

# (CHECK THE PATHs ARE CORRECT)
train_data_path = "../../dataset/train.csv"
test_data_path = "../../dataset/test.csv"
print("Please, check the paths to the train file and to the test file are correct:")
print(train_data_path)
print(test_data_path)

# 1. load the train data 
print("Loading train data...")
y_loaded, x_loaded, ids_te = load_csv_data(train_data_path, sub_sample=False)
y_loaded = y_loaded.reshape((-1, 1))
print("Train data loaded")

# 2. clean the train data and concatenate to each dataset its log
xs, ys = clean_input_data(x_loaded.copy(), y_loaded.copy(), corr=1, dimension_expansion=5, bool_col=True)
for jet in range(4): # set -1 to 0 
    ys[jet][ys[jet]== -1] = 0 
    
xs, mean_log, std_log = concatenate_log(xs.copy())
print("Train data cleaned")

# 3. Build the polynomials (one for each one of the 4 datasets)
degree = 2
txs = [None]*4
for jet in range(4):
    txs[jet] = build_poly(xs[jet], degree)
print("The train polynomials have been built")
    
# 4. Set the array of gammas for the logistic regression
gamma_constants = [1e-5, 1e-6] # one ofr the degree 1 and one for the degree 2
gammas = [None]*4
for jet in range(4):
    ncolumns = xs[jet].shape[1]
    gammas[jet] = np.concatenate([[gamma_constants[0]]] + [ncolumns*[g] for g in gamma_constants])\
        .reshape((-1,1))

# 5. run the logistic regression on the four datasets
def logistic_regression_on_jet(jet):
    y = ys[jet]
    tx = txs[jet]
    
    initial_w = weigths[jet] #np.zeros((tx.shape[1], 1)) #
    max_iters = 2000
    return logistic_regression(y, tx, initial_w, max_iters, gammas[jet])
   
print("Starting to run the logistic regression:")
weigths = [np.zeros((tx.shape[1])) for tx in txs]*4
for jet in range(4):
    print("-Logistic regression on dataset", str(jet) + "/3")
    _, weigths[jet] = logistic_regression_on_jet(jet)

# 6. Load the test data 
print("Loading test data...")
y_te_loaded, x_te_loaded, ids_te_loaded = load_csv_data(test_data_path, sub_sample=False)
print("Test data loaded")

# 6. Clean and append the log (in the same exact way we did with the train set)
x_te, ids_te = clean_input_data(x_te_loaded.copy(), ids_te_loaded.copy(), corr=1, dimension_expansion=5, bool_col=True)

x_te, _, _ = concatenate_log(x_te.copy(), mean_log=mean_log, std_log=std_log)
print("Test data cleaned.")

# 7. Build the polynomials
tx_te = []
for jet in range(4):
    tx_te.append(build_poly(x_te[jet], degree))
print("The test polynomials have been built")

# 8. Predict and concatenate the predicitions
y_te_pred = []
for jet in range(4):
    y_te_pred.append(predict_labels(weigths[jet], tx_te[jet]))
    
for jet in range(4):
    ids_te[jet] = ids_te[jet].reshape((-1, 1))
y_pred = np.row_stack([y_te_pred[0], y_te_pred[1], y_te_pred[2], y_te_pred[3]])
ids = np.row_stack([ids_te[0], ids_te[1], ids_te[2], ids_te[3]])

print("I predicted ", str((y_pred==-1).sum()), "-1s and ", str((y_pred==1).sum()), "1s")

# 9. Store the predictions 
sub_file_name = "predictions"
create_csv_submission(ids, y_pred, sub_file_name)
print("Prediction store in file" + sub_file_name)


Please, check the paths to the train file and to the test file are correct:
../../dataset/train.csv
../../dataset/test.csv
Loading train data...
Train data loaded
Capping the outliers
Jet 0 useless columns dropped: [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28, 29]
Jet 1 useless columns dropped: [4, 5, 6, 12, 22, 26, 27, 28]
Jet 2 useless columns dropped: [22]
Jet 3 useless columns dropped: [22]
Train data cleaned
The train polynomials have been built
Starting to run the logistic regression:
-Logistic regression on dataset 0/3
Gradient Descent(0/1999): loss=64268.2203846304, prediction ratio=0.7449280874360694
Gradient Descent(50/1999): loss=35844.15651465327, prediction ratio=0.8429033258935273
Gradient Descent(100/1999): loss=35324.298035468426, prediction ratio=0.8453054157116692
Gradient Descent(150/1999): loss=35109.24124066348, prediction ratio=0.8456256943540881
Gradient Descent(200/1999): loss=34984.4930347388, prediction ratio=0.84616616456317
Gradient Descent(250/1999): loss=34898

Gradient Descent(200/1999): loss=18774.960910800448, prediction ratio=0.8341372397229004
Gradient Descent(250/1999): loss=18689.362897836385, prediction ratio=0.834951070882709
Gradient Descent(300/1999): loss=18631.055411254645, prediction ratio=0.8352488139899561
Gradient Descent(350/1999): loss=18589.040887528543, prediction ratio=0.8357847515830008
Gradient Descent(400/1999): loss=18557.297763666327, prediction ratio=0.8360427956092816
Gradient Descent(450/1999): loss=18532.31625697155, prediction ratio=0.8363008396355625
Gradient Descent(500/1999): loss=18511.95948566297, prediction ratio=0.8364794854999107
Gradient Descent(550/1999): loss=18494.87994962761, prediction ratio=0.8365985827428095
Gradient Descent(600/1999): loss=18480.200783776116, prediction ratio=0.836955874471506
Gradient Descent(650/1999): loss=18467.334310653914, prediction ratio=0.8367970781476409
Gradient Descent(700/1999): loss=18455.8755467097, prediction ratio=0.8370352726334386
Gradient Descent(750/1999): 